## Module 1 Homework

In this homework, we're going to download finance data from various sources and make simple calculations/analysis.

---

In [48]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


### Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). 
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

---


In [49]:
# Get dates
end = date.today()
start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Year = {end.year}; month= {end.month}; day={end.day}')
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=23
Period for indexes: 1954-04-23 to 2024-04-23 


In [50]:
# Download Real Gross Domestic Product (GDPC1)
# https://fred.stlouisfed.org/series/GDPC1
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)

In [51]:
gdpc1['gdpc1_us_yoy'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1
gdpc1['gdpc1_us_qoq'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(1)-1
gdpc1.tail()

,GDPC1,gdpc1_us_yoy,gdpc1_us_qoq
DATE,,,
2022-10-01,21989.981,0.006517,0.006354
2023-01-01,22112.329,0.017179,0.005564
2023-04-01,22225.350,0.023825,0.005111
2023-07-01,22490.692,0.029269,0.011939
2023-10-01,22679.255,0.031345,0.008384


In [52]:
# Filter data for 2023
gdpc1_2023 = gdpc1.loc['2023']

# Calculate average YoY growth in 2023
average_yoy_growth_2023 = gdpc1_2023['gdpc1_us_yoy'].mean()
average_yoy_growth_2023

0.02540443392867625

In [53]:
# Round the result to 1 decimal
# Convert to percentage and round
average_yoy_growth_2023 = round(average_yoy_growth_2023 * 100, 1)

print(f'Year = {end.year}; month= {end.month}; day={end.day}')
print(f'Period for indexes: {start} to {end} ')
print(f'Average YoY growth in 2023: {average_yoy_growth_2023}%')

Year = 2024; month= 4; day=23
Period for indexes: 1954-04-23 to 2024-04-23 
Average YoY growth in 2023: 2.5%


### Question 2. [Macro] Inverse "Treasury Yield"

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**


Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2,
 https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.
    
(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

---

In [54]:
#Download DGS2 and DGS10 from FRED
start = date(2000, 1, 1)
dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)

In [55]:
#Join dgs2 and dgs10
df = pd.concat([dgs2, dgs10], axis=1)
df

,DGS2,DGS10
DATE,,
2000-01-03,6.38,6.58
2000-01-04,6.30,6.49
2000-01-05,6.38,6.62
2000-01-06,6.35,6.57
2000-01-07,6.31,6.52
...,...,...
2024-04-16,4.97,4.67
2024-04-17,4.93,4.59
2024-04-18,4.98,4.64


In [56]:
# Calculate the difference between DGS2 and DGS10
df['difference'] = df['DGS10'] - df['DGS2']
df

,DGS2,DGS10,difference
DATE,,,
2000-01-03,6.38,6.58,0.20
2000-01-04,6.30,6.49,0.19
2000-01-05,6.38,6.62,0.24
2000-01-06,6.35,6.57,0.22
2000-01-07,6.31,6.52,0.21
...,...,...,...
2024-04-16,4.97,4.67,-0.30
2024-04-17,4.93,4.59,-0.34
2024-04-18,4.98,4.64,-0.34


In [57]:
# Find the min value of (dgs10-dgs2)
# Round to 1 digit after the decimal point

min_difference = round(df['difference'].min(), 1)
min_difference

-1.1

In [58]:
print(f"The min value of (dgs10-dgs2) since 2000-01-01 is: {min_difference}")

The min value of (dgs10-dgs2) since 2000-01-01 is: -1.1


### Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

---

In [59]:
#Define dates
start_date = "2019-04-09"
end_date = "2024-04-09"

#Download S&P 500
sp500 = yf.download(tickers="^GSPC", start=start_date, end=end_date, interval="1d")

#Download IPC Mexico
ipc_mex = yf.download(tickers="^MXX", start=start_date, end=end_date, interval="1d")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [60]:
# Calculate growth
sp500_growth = (sp500['Close'].iloc[-1] / sp500['Close'].iloc[0]) - 1

ipc_mex_growth= (ipc_mex['Close'].iloc[-1] / ipc_mex['Close'].iloc[0]) - 1

In [61]:
#  Select the higher growing index and write down the growth in %

if sp500_growth > ipc_mex_growth:
    print(f"S&P 500 index had the higher growth at {round(sp500_growth * 100)}%.")
else:
    print(f"IPC Mexico index had the higher growth at {round(ipc_mex_growth * 100)}%.")

S&P 500 index had the higher growth at 81%.


### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks


**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**


Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value.
Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

---

In [66]:
# Download data 

# 2222.SR
SAR = yf.download(tickers = "2222.SR", period = "max", interval = "1d")

#BRK-B
BRK = yf.download(tickers = "BRK-B", period = "max", interval = "1d")

#AAPL
AAPL = yf.download(tickers = "AAPL", period = "max", interval = "1d")

#MSFT
MSFT = yf.download(tickers = "MSFT", period = "max", interval = "1d")

#GOOG
GOOG = yf.download(tickers = "GOOG", period = "max", interval = "1d")

#JPM
JPM = yf.download(tickers = "JPM", period = "max", interval = "1d")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [65]:
SAR

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-11,29.090908,29.090908,29.090908,29.090908,25.485229,38289394
2019-12-12,31.983471,31.983471,29.752066,30.413223,26.643656,505692621
2019-12-15,30.661158,30.991735,30.413223,30.909090,27.078062,98349281
2019-12-16,30.991735,31.487602,30.991735,31.404959,27.512466,105973907
2019-12-17,31.446280,31.528925,30.991735,31.198347,27.331469,142672245
...,...,...,...,...,...,...
2024-04-17,30.049999,30.150000,29.900000,29.950001,29.950001,16440263
2024-04-18,30.000000,30.049999,29.850000,29.900000,29.900000,7985988
2024-04-21,30.000000,30.200001,29.900000,30.049999,30.049999,8988623


### Question 5. [Stocks] Dividend Yield
**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximum value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

---

### Question 6. [Exploratory] Investigate new metrics

**Free text answer**

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

---

### Question 7. [Exploratory] Time-driven strategy description around earnings releases

**Free text answer**

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08). 

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.



## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw01